In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

"""  1. cap-shape:                bell=b,conical=c,convex=x,flat=f,
                                  knobbed=k,sunken=s
     2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
     3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r,
                                  pink=p,purple=u,red=e,white=w,yellow=y
     4. bruises?:                 bruises=t,no=f
     5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f,
                                  musty=m,none=n,pungent=p,spicy=s
     6. gill-attachment:          attached=a,descending=d,free=f,notched=n
     7. gill-spacing:             close=c,crowded=w,distant=d
     8. gill-size:                broad=b,narrow=n
     9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g,
                                  green=r,orange=o,pink=p,purple=u,red=e,
                                  white=w,yellow=y
    10. stalk-shape:              enlarging=e,tapering=t
    11. stalk-root:               bulbous=b,club=c,cup=u,equal=e,
                                  rhizomorphs=z,rooted=r,missing=?
    12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
    13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
    14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                                  pink=p,red=e,white=w,yellow=y
    15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                                  pink=p,red=e,white=w,yellow=y
    16. veil-type:                partial=p,universal=u
    17. veil-color:               brown=n,orange=o,white=w,yellow=y
    18. ring-number:              none=n,one=o,two=t
    19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l,
                                  none=n,pendant=p,sheathing=s,zone=z
    20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r,
                                  orange=o,purple=u,white=w,yellow=y
    21. population:               abundant=a,clustered=c,numerous=n,
                                  scattered=s,several=v,solitary=y
    22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p,
                                  urban=u,waste=w,woods=d


"""


In [ ]:
x="""class
cap-shape
cap-surface
cap-color
bruises
odor
gill-attachment
gill-spacing
gill-size
gill-color
stalk-shape
stalk-root
stalk-surface-above-ring
stalk-surface-below-ring
stalk-color-above-ring
stalk-color-below-ring
veil-type
veil-color
ring-number
ring-type
spore-print-color
population
habitat""".split("\n")

In [ ]:
col=[i.split("/")[0].strip() for i in x]


In [ ]:
ds=pd.read_csv("mushroom.csv",names=col,header=None)
ds.head()

# EDA

In [ ]:
ds.shape

In [ ]:
ds.columns

In [ ]:
ds.dtypes

In [ ]:
ds.describe()

In [ ]:
ds.info()

In [ ]:
ds.isnull() # checking for any null values

In [ ]:
ds.isnull().sum() # null  values column wise counts

In [ ]:
# as per data description dataset has ? in 2480 places . so replacing it wih NaN
df = ds.replace({'?':np.NaN})
df

In [ ]:
# column "stalk-root " has NaN. replacing it with missing (m)
df['stalk-root'] = df['stalk-root'].replace(np.NaN,'m')
print(df['stalk-root'].value_counts())

In [ ]:
# dataset has column "veil-type" with value "p" i.e partial only not "u" i.e universal
df["veil-type"].value_counts()

#as the dataset has all nominal values so changnig them to numerical values for further model tuning

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Categorical boolean mask
col_mask = df.dtypes==object
# filter categorical columns using mask and turn it into a list
cols = df.columns[col_mask].tolist()
le=LabelEncoder()
# apply le on categorical feature columns
df[cols] = df[cols].apply(lambda col: le.fit_transform(col))
df[cols].head(10)

In [ ]:
df.head()

In [ ]:
#df.bruises.unique()
df["ring-number"].unique()

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
dfcor=df.corr()

In [ ]:
dfcor

In [ ]:
sns.heatmap(dfcor,annot=True)

In [ ]:
# bivariate analysis to check X variable with Y 1 for poisonous and 0 for edible

In [ ]:
plt.scatter(df["class"],df["gill-size"])

In [ ]:
plt.scatter(df["class"],df["ring-number"])

In [ ]:
plt.scatter(df["class"],df["ring-type"])

In [ ]:
plt.scatter(df["class"],df["gill-color"])

In [ ]:
plt.scatter(df["class"],df["population"])

In [ ]:
plt.scatter(df["class"],df["habitat"])

In [ ]:
# veil type is always showing p (partial) in dataset thats having value 0 and no value for u (universal)always.
# also, thats why its corr() is NaN
plt.scatter(df["class"],df["veil-type"])

#it shows that 51% mushrooms are edible and 48% are poisonous

In [ ]:

df['class'].value_counts(normalize=True)

In [ ]:
# there are no outliers .
# No droping of columns
#so no more  data cleaning is to be done. only replacing of ? 

In [ ]:
#Drop target feature
x = df.drop(columns='class') 
y = df['class']

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
df.skew()

In [ ]:
# removing skewness by transforming data
from sklearn.preprocessing import power_transform

In [ ]:
x=power_transform(x,method='yeo-johnson')

#data modelling

Logistic REgression

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.22,random_state=43)

In [ ]:
lr=LogisticRegression()

In [ ]:
lr.fit(x_train,y_train)
lr.score(x_train,y_train)
predlr=lr.predict(x_test)
print("accuracy score",accuracy_score(y_test,predlr))
print("confusion matrix\n",confusion_matrix(y_test,predlr))
print("classification report\n",classification_report(y_test,predlr))

In [ ]:
# max_depth 1 to 5 is giving increasingly high accuracy score

In [ ]:
dtc=DecisionTreeClassifier(max_depth=5)
dtc.fit(x_train,y_train)
dtc.score(x_train,y_train)
preddtc=dtc.predict(x_test)
print("accuracy score",accuracy_score(y_test,preddtc))
print("confusion matrix\n",confusion_matrix(y_test,preddtc))
print("classification report\n",classification_report(y_test,preddtc))

In [ ]:
rfc=RandomForestClassifier(n_estimators=100,max_depth=5)
rfc.fit(x_train,y_train)
rfc.score(x_train,y_train)
predrfc=rfc.predict(x_test)
print("accuracy score",accuracy_score(y_test,predrfc))
print("confusion matrix\n",confusion_matrix(y_test,predrfc))
print("classification report\n",classification_report(y_test,predrfc))

In [ ]:
gnb=GaussianNB()
gnb.fit(x_train,y_train)
gnb.score(x_train,y_train)
predgnb=gnb.predict(x_test)
print("accuracy score",accuracy_score(y_test,predgnb))
print("confusion matrix\n",confusion_matrix(y_test,predgnb))
print("classification report\n",classification_report(y_test,predgnb))

#Conclusion- RandomForestClassifier is giving the highest accuracy score with .98

In [ ]:
# saving the prediction model

import pickle
filename="MushroomClass.pkl"
pickle.dump(predrfc,open(filename,'wb'))